In [11]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()

df = pd.read_csv('oecd_data.csv')

liberal = ['AUS','CAN','USA','GBR','IRL','NZL','CHE']
conservative = ['DEU','AUT','BEL','FRA','NLD','ITA','JPN']
socdem = ['SWE','DNK','NOR','FIN']

df['regime'] = df['iso'].replace(liberal+conservative+socdem,7*['liberal']+7*['conservative']+4*['social democratic'])

df = df[df['regime'].isin(['liberal','conservative','social democratic'])]
df['regime'] = df['regime'].astype('category')
df['regime'] = df['regime'].cat.reorder_categories(['liberal','conservative','social democratic'])


df.head()

,iso,country,measure_id,measure_name,age_id,age_name,methodology_id,methodology_name,year,value,regime
0,AUS,Australia,GINI,"Gini (disposable income, post taxes and transf...",TOT,Total population,METH2012,New income definition since 2012,2012,0.326,liberal
1,AUS,Australia,GINI,"Gini (disposable income, post taxes and transf...",TOT,Total population,METH2012,New income definition since 2012,2014,0.337,liberal
2,AUS,Australia,GINI,"Gini (disposable income, post taxes and transf...",TOT,Total population,METH2012,New income definition since 2012,2016,0.330,liberal
3,AUS,Australia,GINI,"Gini (disposable income, post taxes and transf...",TOT,Total population,METH2011,Income definition until 2011,2004,0.315,liberal
4,AUS,Australia,GINI,"Gini (disposable income, post taxes and transf...",TOT,Total population,METH2011,Income definition until 2011,2008,0.336,liberal


In [19]:
table = pd.pivot_table(df.loc[df['age_id']=='TOT'],index=['iso','year','regime','methodology_id'],columns='measure_id',values='value')
table = table.reset_index(2).reset_index(2)
table.head()

measure_id methodology_id   regime     CPI2010   CPI2015  ECTOTAL   GINI  \
iso year                                                                   
AUS 2004         METH2011  liberal   84.079081  75.04063  29928.0  0.315   
    2008         METH2011  liberal   95.473460  85.21012  42435.0  0.336   
    2010         METH2011  liberal  100.000000  89.25006  43571.0  0.334   
    2012         METH2011  liberal  105.124871  93.82401  47924.0  0.324   
    2012         METH2012  liberal  105.124871  93.82401  47924.0  0.326   

measure_id  GINIB  GINIG        HHD   INCAC1  ...   SHA4   SHA5   SHA6   SHA7  \
iso year                                      ...                               
AUS 2004    0.465    NaN  7735778.0  30128.0  ...  0.148  0.166  0.068  0.045   
    2008    0.468    NaN  8077302.0  41753.0  ...  0.145  0.175  0.068  0.050   
    2010    0.469    NaN  8398520.0  44389.0  ...  0.142  0.177  0.068  0.049   
    2012    0.460    NaN  8630428.0  48217.0  ...  0.140  0.179  0.072  0.051   
    2012    0.463  0.368  8630428.0  47782.0  ...  0.140  0.179  0.072  0.051   

measure_id   STDG  TACTOTAL  TRPCTOTAL  TRPERCTOTAL  TRPOTCTOTAL  TRRCTOTAL  
iso year                                                                     
AUS 2004    0.004   -7606.0        NaN          NaN          NaN        NaN  
    2008    0.004   -9750.0        NaN          NaN          NaN        NaN  
    2010    0.006   -8999.0        NaN          NaN          NaN        NaN  
    2012    0.004  -10358.0        NaN          NaN          NaN        NaN  
    2012    0.005  -10358.0   -11128.0          NaN       -770.0     9136.0  

[5 rows x 60 columns]

In [36]:
import statsmodels.formula.api as smf

res = smf.ols('GINI ~ GINIB + regime + methodology_id',data=table).fit(cov_type='HC1')
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   GINI   R-squared:                       0.627
Model:                            OLS   Adj. R-squared:                  0.621
Method:                 Least Squares   F-statistic:                     184.2
Date:                Thu, 23 Jan 2020   Prob (F-statistic):           2.02e-73
Time:                        10:22:54   Log-Likelihood:                 616.75
No. Observations:                 256   AIC:                            -1223.
Df Residuals:                     251   BIC:                            -1206.
Df Model:                           4                                         
Covariance Type:                  HC1                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       0.2470      0.016     15.795      0.000       0.216       0.278
regime[T.conservative]         -0.0367      0.004    -10.189      0.000      -0.044      -0.030
regime[T.social democratic]    -0.0637      0.002    -25.894      0.000      -0.069      -0.059
methodology_id[T.METH2012]      0.0009      0.003      0.316      0.752      -0.005       0.006
GINIB                           0.1696      0.036      4.730      0.000       0.099       0.240
==============================================================================
Omnibus:                       11.934   Durbin-Watson:                   0.217
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               12.259
Skew:                           0.500   Prob(JB):                      0.00218
Kurtosis:                       3.386   Cond. No.                         35.5
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [35]:
res1 = smf.ols('PVT6A ~ PVT6B + regime + methodology_id',data=table).fit()
res1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  PVT6A   R-squared:                       0.419
Model:                            OLS   Adj. R-squared:                  0.409
Method:                 Least Squares   F-statistic:                     45.54
Date:                Thu, 23 Jan 2020   Prob (F-statistic):           8.62e-29
Time:                        10:20:27   Log-Likelihood:                 608.35
No. Observations:                 258   AIC:                            -1207.
Df Residuals:                     253   BIC:                            -1189.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       0.1775      0.009     19.234      0.000       0.159       0.196
regime[T.conservative]         -0.0251      0.003     -7.370      0.000      -0.032      -0.018
regime[T.social democratic]    -0.0488      0.004    -12.969      0.000      -0.056      -0.041
methodology_id[T.METH2012]     -0.0002      0.003     -0.057      0.955      -0.006       0.006
PVT6B                           0.0400      0.027      1.477      0.141      -0.013       0.093
==============================================================================
Omnibus:                       11.810   Durbin-Watson:                   0.208
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               12.679
Skew:                           0.541   Prob(JB):                      0.00177
Kurtosis:                       2.900   Cond. No.                         26.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
from linearmodels import BetweenOLS

mod = BetweenOLS.from_formula('GINI ~ 1 + GINIB + regime + methodology_id',data=table.dropna(subset=['GINI','GINIB'])).fit(cov_type='robust')
mod.summary

Dep. Variable:,GINI,R-squared:,0.6989
Estimator:,BetweenOLS,R-squared (Between):,0.6989
No. Observations:,18,R-squared (Within):,-10.781
Date:,"Thu, Jan 23 2020",R-squared (Overall):,0.1207
Time:,10:09:59,Log-likelihood,45.469
Cov. Estimator:,Robust,,
,,F-statistic:,7.5444
Entities:,18,P-value,0.0023
Avg Obs:,14.222,Distribution:,"F(4,13)"
Min Obs:,6.0000,,
Max Obs:,23.000,F-statistic (robust):,19.912


In [34]:
mod1 = BetweenOLS.from_formula('PVT6A ~ 1 + regime + PVT6B',data=table.dropna(subset=['PVT6A','PVT6B'])).fit(cov_type='robust')
mod1.summary

Dep. Variable:,PVT6A,R-squared:,0.4356
Estimator:,BetweenOLS,R-squared (Between):,0.4356
No. Observations:,18,R-squared (Within):,-0.1504
Date:,"Thu, Jan 23 2020",R-squared (Overall):,0.3903
Time:,10:20:04,Log-likelihood,41.938
Cov. Estimator:,Robust,,
,,F-statistic:,3.6017
Entities:,18,P-value,0.0407
Avg Obs:,14.333,Distribution:,"F(3,14)"
Min Obs:,6.0000,,
Max Obs:,23.000,F-statistic (robust):,6.0786
